实现 role-play 对话数据生成工具，要求包含下列功能：
1.基于一段文本（自己找一段文本，复制到提示词就可以了，比如你可以从小说中选取一部分文本，注意文本要用 markdown 格式）生成角色人设，可借助 ChatGLM 实现。
2.给定两个角色的人设，调用 CharacterGLM 交替生成他们的回复。
3.将生成的对话数据保存到文件中

In [5]:
%pip install langchain transformers torch accelerate
%pip install sentencepiece
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import json
from langchain.llms import ChatGLM
from transformers import AutoModel, AutoTokenizer
import torch
import re

# 1. 基于文本生成角色人设
def generate_character_profiles(text, chatglm_model_path):
    """
    使用 ChatGLM 生成角色人设。

    Args:
        text: 输入的文本（markdown格式）。
        chatglm_model_path: ChatGLM 模型路径。

    Returns:
        一个包含角色人设的字典。
    """
    llm = ChatGLM(
        model_path=chatglm_model_path,
        temperature=0.8,
        top_p=0.8,
        max_token=1024
    )
    prompt = f"""请根据以下文本，提取出2个主要角色，并为他们分别生成详细的角色人设，包括姓名，年龄，性格特点，背景故事等。请使用markdown格式输出。
    文本内容:\n{text}
    """
    response = llm(prompt)
    print(f"ChatGLM Response:\n{response}")
    
    # 使用正则表达式提取角色人设
    profiles = {}
    role_pattern = re.compile(r"### 角色(.*?)\n(.*?)###", re.DOTALL)
    matches = role_pattern.findall(response)
    for i, match in enumerate(matches):
        role_name = match[0].strip()
        role_profile = match[1].strip()
        profiles[f"role{i+1}"]={"name":role_name, "profile":role_profile}

    return profiles


# 2. 使用 CharacterGLM 生成对话
def generate_dialogue(character_profiles, characterglm_model_path):
    """
    使用 CharacterGLM 交替生成角色回复。

    Args:
        character_profiles: 包含角色人设的字典。
        characterglm_model_path: CharacterGLM 模型路径。

    Returns:
        生成的对话列表，每个元素为一个字典，包含角色和回复。
    """
    tokenizer = AutoTokenizer.from_pretrained(characterglm_model_path, trust_remote_code=True)
    model = AutoModel.from_pretrained(characterglm_model_path, trust_remote_code=True).half().cuda()
    
    role1_name = character_profiles["role1"]["name"]
    role1_profile = character_profiles["role1"]["profile"]
    role2_name = character_profiles["role2"]["name"]
    role2_profile = character_profiles["role2"]["profile"]
    
    
    history = []
    
    dialogues = []
    
    for i in range(10):
        if i % 2 == 0:
            
            prompt = f"""{role1_name}: {role1_profile},请根据你的身份和经历，接下来说一句话，以下是之前的对话历史：{history}
            """
            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
            outputs = model.generate(**inputs, max_new_tokens=128, do_sample=True, top_p=0.8, temperature=0.8)
            response = tokenizer.decode(outputs[0], skip_special_tokens=True).split(":")[-1].strip()
            
            dialogues.append({"role": role1_name, "response": response})
            history.append(f"{role1_name}: {response}")
        else:
            prompt = f"""{role2_name}: {role2_profile},请根据你的身份和经历，接下来说一句话，以下是之前的对话历史：{history}
            """
            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
            outputs = model.generate(**inputs, max_new_tokens=128, do_sample=True, top_p=0.8, temperature=0.8)
            response = tokenizer.decode(outputs[0], skip_special_tokens=True).split(":")[-1].strip()
            
            dialogues.append({"role": role2_name, "response": response})
            history.append(f"{role2_name}: {response}")

    return dialogues


# 3. 将对话数据保存到文件
def save_dialogue_data(dialogues, filename="dialogue_data.json"):
    """
    将对话数据保存到 JSON 文件。

    Args:
        dialogues: 对话列表。
        filename: 保存的文件名。
    """
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(dialogues, f, indent=4, ensure_ascii=False)
    print(f"对话数据已保存到 {filename}")



In [ ]:
    # 示例文本
    example_text = """
    # 第一章

    风雪交加，寒冷刺骨。
    
    在一家破旧的酒馆里，一个孤独的旅人，名叫李明，正独自喝着闷酒。他的脸上布满了风霜，眼神中透露着一丝疲惫。
    
    突然，酒馆的门被推开，一个衣着华丽的女子，名叫林雪，走了进来。她身穿一件白色的裘皮大衣，显得格外耀眼。她的脸上带着一丝高傲，似乎对周围的一切都漠不关心。
    
    “老板，来一杯热酒吧。”林雪冷冷地说道。
    """
    
    chatglm_model_path = "THUDM/chatglm2-6b"  # 请替换为您的 ChatGLM 模型路径
    characterglm_model_path = "THUDM/characterglm-6b" # 请替换为您的 CharacterGLM 模型路径
    
    # 1. 生成角色人设
    character_profiles = generate_character_profiles(example_text, chatglm_model_path)
    print("Character Profiles:", character_profiles)
    
    # 2. 生成对话
    dialogues = generate_dialogue(character_profiles, characterglm_model_path)
    print("Dialogues:", dialogues)

    # 3. 保存对话数据
    save_dialogue_data(dialogues)